# Averaged Artifact Substraction with EEG Data
## First step prepare Jupyter Notebook

In [ ]:
import os
from facet.facet import facet
from loguru import logger
import sys

bids_path = "F:\EEG Datasets\openneuro\FMRIWITHMOTION"
export_bids_path = bids_path

# configure logger
logger.remove()
logger.add(sys.stderr, level="DEBUG")
logger.add("facet.log", level="DEBUG")

window_size = 30
upsampling_factor = 1
artifact_to_trigger_offset_in_seconds = -0.038
relative_window_position = -0.5
moosmann_motion_threshold = 0.8
event_id_description_pairs = {"trigger": 1}
regex_trigger_annotation_filter = r"\bResponse\b"  # Annotations with the description 'trigger' are considered as triggers
unwanted_bad_channels = [
    "EKG",
    "EMG",
    "EOG",
    "ECG",
]  # Channels with these names are considered as bad channels and not considered in the processing
evaluation_measures = ["SNR", "RMS", "RMS2", "MEDIAN"]

f = facet()

## Second Step import EEG Data

In [ ]:
f.import_eeg(
    path=bids_path,
    fmt="bids",
    upsampling_factor=upsampling_factor,
    artifact_to_trigger_offset=artifact_to_trigger_offset_in_seconds,
    bads=["EKG", "EMG", "EOG", "ECG"],
    subject="xp101",
    session=None,
    task="eegfmriNF",
)
f.plot_eeg(title="after import")

## Third Step Preprocessing

In [ ]:
f.pre_processing()

## Fourth Step find triggers

In [ ]:
f.find_triggers(regex_trigger_annotation_filter)  # Using Niazys data
f.find_missing_triggers()
f.align_triggers(0)

## Fourth step apply AAS

In [ ]:
f.calc_matrix_aas()

## Fifth step Remove Artifacts

In [ ]:
f.remove_artifacts(plot_artifacts=True)

## Sixth step Post-Processing

In [ ]:
f.post_processing()

## Seventh display the processed eeg data

In [ ]:
f.plot_eeg(title="after lowpass", start=29)

## Eighth Evaluate Results and compare with other Methods

In [ ]:
eeg_python = f.get_eeg()
# Matrix will result in >1.0 RMS2 because of python mne inaccuracies
f.add_to_evaluate(eeg_python, name="numpy_matrix")

# moosmann
f.import_eeg(
    path=bids_path,
    fmt="bids",
    artifact_to_trigger_offset=artifact_to_trigger_offset_in_seconds,
    upsampling_factor=upsampling_factor,
    bads=unwanted_bad_channels,
    subject="xp101",
    session=None,
    task="eegfmriNF",
)
#print current path
print(os.getcwd())
f.pre_processing()
f.find_triggers(regex_trigger_annotation_filter)
f.find_missing_triggers()
f.align_triggers(0)
f.calc_matrix_motion(
    file_path="../datasets/headmotiondata_209s.tsv", threshold=moosmann_motion_threshold
)
f.remove_artifacts()
f.post_processing()
f.add_to_evaluate(f.get_eeg(), name="Moosmann")

results = f.evaluate(plot=True, measures=["SNR", "RMS", "RMS2", "MEDIAN"])
logger.info(results)

# f.export_eeg(fmt="bids", event_id=event_id_description_pairs, path=export_bids_path)